In [ ]:
pip install transformers

In [ ]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

--2024-02-12 13:38:14--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M  --.-KB/s    in 0.09s   

2024-02-12 13:38:17 (429 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2024-02-12 13:38:17--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [ ]:
import os
import json
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import BertTokenizerFast

In [ ]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('KorQuAD_v1.0_train.json')
val_contexts, val_questions, val_answers = read_squad('KorQuAD_v1.0_dev.json')

In [ ]:
print('훈련 데이터의 본문 개수 :', len(train_contexts))
print('훈련 데이터의 질문 개수 :', len(train_questions))
print('훈련 데이터의 답변 개수 :', len(train_answers))
print('테스트 데이터의 본문 개수 :', len(val_contexts))
print('테스트 데이터의 질문 개수 :', len(val_questions))
print('테스트 데이터의 답변 개수 :', len(val_answers))

훈련 데이터의 본문 개수 : 60407
훈련 데이터의 질문 개수 : 60407
훈련 데이터의 답변 개수 : 60407
테스트 데이터의 본문 개수 : 5774
테스트 데이터의 질문 개수 : 5774
테스트 데이터의 답변 개수 : 5774


In [ ]:
print('첫번째 샘플의 본문')
print('-----------------')
print(train_contexts[0])

첫번째 샘플의 본문
-----------------
1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.


In [ ]:
print('첫번째 샘플의 질문')
print('-----------------')
print(train_questions[0])

첫번째 샘플의 질문
-----------------
바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?


In [ ]:
print('첫번째 샘플의 답변')
print('-----------------')
print(train_answers[0])

첫번째 샘플의 답변
-----------------
{'text': '교향곡', 'answer_start': 54}


In [ ]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        # 뒤에 공백이 있다면 제거. ex)'1990년대 ' -> '1990년대'
        answer['text'] = answer['text'].rstrip()

        # 시작 인덱스에 정답의 길이를 더하여 종료 인덱스 계산
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        assert context[start_idx:end_idx] == gold_text, "end_index 계산에 에러가 있습니다."
        answer['answer_end'] = end_idx

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [ ]:
print('첫번째 샘플의 답변')
print('-----------------')
print(train_answers[0])

첫번째 샘플의 답변
-----------------
{'text': '교향곡', 'answer_start': 54, 'answer_end': 57}


In [ ]:
train_contexts[0][54]

'교'

In [ ]:
train_contexts[0][55]

'향'

In [ ]:
train_contexts[0][56]

'곡'

In [ ]:
train_contexts[0][54:57]

'교향곡'

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True, max_length=256)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [ ]:
train_encodings[0]

Encoding(num_tokens=256, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
print(train_encodings[0].ids)

[2, 13934, 2236, 2440, 27982, 2259, 21310, 2079, 11994, 3791, 2069, 3790, 1508, 2088, 636, 3800, 2170, 3717, 2052, 9001, 8345, 4642, 2200, 3689, 3657, 2079, 19282, 2069, 1363, 2370, 2259, 936, 2069, 554, 2259, 2062, 18, 1504, 4342, 27982, 2259, 13934, 2196, 2440, 2170, 1195, 23260, 6233, 17370, 2113, 2165, 2069, 809, 1, 3706, 2052, 2181, 8642, 2145, 7334, 2170, 4983, 2371, 4007, 1065, 5917, 2386, 2559, 4443, 2138, 4026, 2259, 11994, 3791, 2079, 15864, 2170, 5487, 2371, 4683, 3605, 18, 3819, 5986, 27135, 1376, 2645, 2203, 2292, 2079, 5872, 2200, 5986, 4152, 2252, 22835, 16706, 2052, 5485, 2205, 2259, 17087, 2079, 19282, 29, 2517, 2069, 881, 2088, 652, 2073, 23404, 2069, 1122, 2886, 13964, 16, 3982, 2052, 9944, 21, 2429, 2170, 11994, 3791, 2079, 1258, 2465, 6233, 24294, 1504, 3967, 2170, 4027, 2052, 5121, 3979, 2069, 18274, 21575, 23548, 575, 2073, 5292, 2085, 7251, 2116, 1415, 2062, 18, 3776, 2079, 942, 2286, 2446, 4196, 2079, 3640, 6509, 636, 2079, 4450, 2170, 10329, 1513, 2259, 575, 7

In [ ]:
print(train_encodings[0].tokens)

['[CLS]', '183', '##9', '##년', '바그너', '##는', '괴테', '##의', '파우', '##스트', '##을', '처음', '읽', '##고', '그', '내용', '##에', '마음', '##이', '끌려', '이를', '소재', '##로', '해서', '하나', '##의', '교향곡', '##을', '쓰', '##려', '##는', '뜻', '##을', '갖', '##는', '##다', '.', '이', '시기', '바그너', '##는', '183', '##8', '##년', '##에', '빛', '독촉', '##으로', '산전', '##수', '##전', '##을', '다', '[UNK]', '상황', '##이', '##라', '좌절', '##과', '실망', '##에', '가득', '##했', '##으며', '메', '##피스', '##토', '##펠', '##레스', '##를', '만나', '##는', '파우', '##스트', '##의', '심경', '##에', '공감', '##했', '##다고', '한다', '.', '또한', '파리', '##에서', '아', '##브', '##네', '##크', '##의', '지휘', '##로', '파리', '음악', '##원', '관현', '##악단', '##이', '연주', '##하', '##는', '베토벤', '##의', '교향곡', '9', '##번', '##을', '듣', '##고', '깊', '##은', '감명', '##을', '받', '##았', '##는데', ',', '이것', '##이', '이듬해', '1', '##월', '##에', '파우', '##스트', '##의', '서', '##곡', '##으로', '쓰여진', '이', '작품', '##에', '조금', '##이', '##라도', '영향', '##을', '끼쳤', '##으리', '##라는', '것', '##은', '의심', '##할', '여지', '##가', '없', '##다', '.', '여기', '##의', '

In [ ]:
print(train_encodings[0].type_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
print(train_encodings[0].attention_mask)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    deleting_list = []

    for i in tqdm(range(len(answers))):
        # 토큰화 전 문자의 인덱스(start, end)로부터 토큰화한 후의 문자의 인덱스(start, end)를 찾아냄.
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # 시작 인덱스가 비정상인 경우. 즉, 본문에 정답이 없는 경우
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
            deleting_list.append(i)

        # 종료 인덱스가 비정상인 경우. 즉, 본문에 정답이 없는 경우
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
            if i not in deleting_list:
              deleting_list.append(i)

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return deleting_list

In [ ]:
deleting_list_for_train = add_token_positions(train_encodings, train_answers)
deleting_list_for_test = add_token_positions(val_encodings, val_answers)

100%|██████████| 5774/5774 [00:00<00:00, 350669.13it/s]


In [ ]:
print('삭제 예정인 훈련 샘플 :', deleting_list_for_train)
print('삭제 예정인 테스트 샘플 :', deleting_list_for_test)

삭제 예정인 훈련 샘플 : [7, 41, 80, 82, 84, 85, 91, 112, 113, 116, 146, 159, 186, 211, 224, 225, 232, 239, 241, 312, 315, 317, 357, 362, 366, 369, 373, 375, 376, 383, 393, 397, 441, 460, 463, 485, 493, 497, 498, 504, 538, 632, 635, 652, 653, 655, 688, 690, 710, 711, 722, 726, 727, 728, 729, 731, 742, 745, 746, 748, 749, 750, 757, 758, 761, 765, 767, 768, 784, 785, 786, 796, 799, 805, 822, 823, 824, 825, 826, 828, 829, 833, 839, 840, 842, 844, 846, 849, 872, 895, 918, 919, 921, 922, 926, 973, 990, 1013, 1014, 1017, 1020, 1037, 1040, 1049, 1051, 1055, 1058, 1064, 1066, 1067, 1073, 1105, 1121, 1165, 1168, 1170, 1171, 1174, 1178, 1196, 1197, 1202, 1208, 1217, 1218, 1219, 1221, 1226, 1229, 1231, 1251, 1253, 1255, 1268, 1299, 1301, 1315, 1318, 1320, 1323, 1345, 1350, 1352, 1355, 1376, 1378, 1379, 1380, 1381, 1383, 1386, 1394, 1395, 1399, 1402, 1405, 1439, 1443, 1467, 1468, 1469, 1543, 1553, 1556, 1557, 1558, 1560, 1561, 1580, 1584, 1608, 1613, 1630, 1654, 1656, 1659, 1661, 1664, 1690, 1694, 1696, 172

In [ ]:
print('761번 샘플의 질문 :',train_questions[761])
print('-'*50)
print('761번 샘플의 기존 원문 :',train_contexts[761])

761번 샘플의 질문 : 소치 팀추월 파이널D에서 여자 팀추월 대표팀의 최종 성적은?
--------------------------------------------------
761번 샘플의 기존 원문 : 2014년 2월 9일 러시아 소치 아들레르 아레나서 열린 소치 동계올림픽 3000m 부문에서 김보름(21)은 4분12초08의 기록으로로 13위를 차지했다. 이날 3조로 경기를 치른 김보름은 21초05로 200m 구간을 통과한 후 2분31초34로 1800m 구간을 지났다. 이후 2200m 구간 통과 순간부터 스피드를 올리며 후반 들어 스퍼트를 올렸고, 결국 하위권이 아닌 중위권 기록을 남겼다. 김보름의 순위인 13위는 지난 2006 토리노 올림픽, 2010 밴쿠버 올림픽 당시 노선영(25-강원도청)이 기록한 19위를 넘어 한국 여자 3000m 부문의 가장 높은 순위다. 5조의 노선영은 4분19초02를 기록했다. 노선영은 200m 구간에서 21초32의 기록으로 지난 이후 속도를 올리지 못한 채로 결승점을 통과했다. 결국 노선영은 전체 26위의 성적을 남기며 경기를 마쳤다. 6조에서 경기를 소화한 양신영(24-전북도청)은 4분23초67을 기록해 이날 대회를 뛴 28명 중 최저의 기록을 남겼다. 한편 이날 대회는 4분00초34의 이레네 부스트(네덜란드)가 2006년 토리노 동계올림픽 이후 8년 만에 금메달을 다시 가져갔다. 대회 2연패를 노린 2위 마르티나 사블리코바(체코-4분01초95)와 3위 올가 그라프(러시아-4분03초47)에 앞선 기록이다. 16일 열린 1500m에서는 네덜란드의 요리엔 테르모르스가 1분53초51의 올림픽 기록으로 금메달을 차지했다. 은메달과 동메달도 네덜란드 선수들이 휩쓸었다. 은메달은 이레인 뷔스트(1분54초09)에게 돌아갔고 동메달은 하를로터 판바이크(1분54초54)가 주인이 됐다. 심지어 4위도 네덜란드 선수인 마리트 리엔스트라(1분56초40)가 차지했다. 김보름은 1분59초78로 21위에 올랐다. 노선영(25-강원도청)은 2분01초

In [ ]:
print('761번 샘플의 기존 정답 :', train_answers[761])

761번 샘플의 기존 정답 : {'text': '8위', 'answer_start': 1050, 'answer_end': 1052}


In [ ]:
print('761번 샘플 전처리 후 :', tokenizer.decode(train_encodings['input_ids'][761]))

761번 샘플 전처리 후 : [CLS] 2014년 2월 9일 러시아 소치 아들레르 아레나서 열린 소치 동계올림픽 3000m 부문에서 김보름 ( 21 ) 은 4분12초08의 기록으로로 13위를 차지했다. 이날 3조로 경기를 치른 김보름은 21초05로 200m 구간을 통과한 후 2분31초34로 1800m 구간을 지났다. 이후 2200m 구간 통과 순간부터 스피드를 올리며 후반 들어 스퍼트를 올렸고, 결국 하위권이 아닌 중위권 기록을 남겼다. 김보름의 순위인 13위는 지난 2006 토리노 올림픽, 2010 밴쿠버 올림픽 당시 노선영 ( 25 - 강원도청 ) 이 기록한 19위를 넘어 한국 여자 3000m 부문의 가장 높은 순위다. 5조의 노선영은 4분19초02를 기록했다. 노선영은 200m 구간에서 21초32의 기록으로 지난 이후 속도를 올리지 못한 채로 결승점을 통과했다. 결국 노선영은 전체 26위의 성적을 남기며 경기를 마쳤다. 6조에서 경기를 소화한 양신영 ( 24 - 전북도청 [SEP] 소치 팀추월 파이널D에서 여자 팀추월 대표팀의 최종 성적은? [SEP]


In [ ]:
def delete_samples(encodings, deleting_list):
  input_ids = np.delete(np.array(encodings['input_ids']), deleting_list, axis=0)
  attention_masks = np.delete(np.array(encodings['attention_mask']), deleting_list, axis=0)
  start_positions = np.delete(np.array(encodings['start_positions']), deleting_list, axis=0)
  end_positions = np.delete(np.array(encodings['end_positions']), deleting_list, axis=0)

  X_data = [input_ids, attention_masks]
  y_data = [start_positions, end_positions]

  return X_data, y_data

In [ ]:
X_train, y_train = delete_samples(train_encodings, deleting_list_for_train)
X_test, y_test = delete_samples(val_encodings, deleting_list_for_test)

In [ ]:
print('-------------삭제 전-------------')
print('훈련 데이터의 샘플의 개수 :', len(train_contexts))
print('테스트 데이터의 샘플의 개수 :', len(val_contexts))
print('-------------삭제 후-------------')
print('훈련 데이터의 샘플의 개수 :', len(X_train[0]))
print('테스트 데이터의 샘플의 개수 :', len(X_test[0]))

-------------삭제 전-------------
훈련 데이터의 샘플의 개수 : 60407
테스트 데이터의 샘플의 개수 : 5774
-------------삭제 후-------------
훈련 데이터의 샘플의 개수 : 54565
테스트 데이터의 샘플의 개수 : 5215


In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertModel, BertTokenizer

class BertForQuestionAnswering(nn.Module):
    # init에는 우리가 사용할 모델의 층들을 선언하는 부분.
    def __init__(self, model_name):
        super(BertForQuestionAnswering, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        # 사용할 출력층에서 사용할 뉴런은 2개. 각각 시작 인덱스 예측과 종료 인덱스 예측에 사용된다.
        self.qa_outputs = nn.Linear(self.bert.config.hidden_size, 2)

    # forward 부분은 init에서 선언한 층들을 연결하는 부분.
    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        # BERT의 마지막 층의 모든 토큰들. outputs[0].shape == (배치 크기, 문장 길이, 768)
        # 예를 들어서 하나의 데이터가 512개의 단어로 구성되어져 있다면 (배치 크기, 512, 768)
        # 그런데 동시에 50개의 데이터를 처리한다면. 즉, 배치 크기가 50이라면 (50, 512, 768)
        sequence_output = outputs[0]

        # 사용할 출력층은 총 뉴런 2개. 각각 시작 인덱스 예측과 종료 인덱스 예측에 사용된다.
        logits = self.qa_outputs(sequence_output)

        # 뉴런 2개를 쪼갠다. 그래서 1개짜리가 2개.
        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        # 시작 인덱스에 대한 다중 클래스 분류 문제.
        # 예를 들어 모델의 최대 길이가 512라면 512개 중 가장 확률값이 높은 1개를 찾는다.
        start_probs = torch.softmax(start_logits, dim=-1)

        # 종료 인덱스에 대한 다중 클래스 분류 문제
        # 예를 들어 모델의 최대 길이가 512라면 512개 중 가장 확률값이 높은 1개를 찾는다.
        end_probs = torch.softmax(end_logits, dim=-1)

        return start_probs, end_probs

model = BertForQuestionAnswering("klue/bert-base")
optimizer = Adam(model.parameters(), lr=5e-5)
loss = nn.CrossEntropyLoss()

In [ ]:
def create_dataset(X_data, y_data):
    """
    입력 데이터(X_data)와 레이블(y_data)를 받아 파이토치의 TensorDataset으로 변환합니다.
    이 함수는 기계 독해 모델 학습을 위한 데이터셋 생성 과정에서 필요한 입력 데이터와 타겟 레이블을 텐서로 변환하고,
    이를 TensorDataset 객체로 묶어 반환합니다. 이렇게 생성된 데이터셋은 DataLoader를 통해 모델 학습에 사용될 수 있습니다.

    Parameters:
    X_data (tuple): 입력 데이터를 담고 있는 튜플. X_data는 (input_ids, attention_masks) 형태로,
                    각각 정수 인코딩된 텍스트와 어텐션 마스크 정보를 포함합니다.
    y_data (tuple): 타겟 레이블 데이터를 담고 있는 튜플. y_data는 (start_positions, end_positions) 형태로,
                    각각 정답이 시작하는 위치와 끝나는 위치의 인덱스 정보를 포함합니다.

    Returns:
    TensorDataset: 입력 데이터와 타겟 레이블을 포함하는 파이토치의 TensorDataset 객체.
    """
    # 입력 데이터(X_data)에서 정수 인코딩된 텍스트와 어텐션 마스크를 추출합니다.
    input_ids, attention_masks = X_data

    # 타겟 레이블(y_data)에서 정답의 시작 위치와 종료 위치를 추출합니다.
    start_positions, end_positions = y_data

    # 추출한 데이터들을 각각 파이토치 텐서로 변환합니다. 데이터 타입은 모두 torch.long으로 설정합니다.
    input_ids = torch.tensor(input_ids, dtype=torch.long)
    attention_masks = torch.tensor(attention_masks, dtype=torch.long)
    start_positions = torch.tensor(start_positions, dtype=torch.long)
    end_positions = torch.tensor(end_positions, dtype=torch.long)

    # 변환된 텐서들을 TensorDataset 객체로 묶어서 반환합니다. 이 객체는 DataLoader와 함께 사용될 수 있습니다.
    dataset = TensorDataset(input_ids, attention_masks, start_positions, end_positions)
    return dataset

In [ ]:
batch_size = 50

In [ ]:
train_dataset = create_dataset(X_train, y_train)
test_dataset = create_dataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [ ]:
54565 / 50

1091.3

In [ ]:
len(train_dataloader)

1092

In [ ]:
def evaluate(model, dataloader, loss, device):
    """
    모델의 성능을 평가하고 데이터로더를 통해 손실 값을 계산합니다. 이 함수는 주어진 데이터로더에 대해 모델을 평가 모드로 설정하고,
    각 배치에 대한 손실을 계산하여 평균 손실을 반환합니다.

    Parameters:
    model (torch.nn.Module): 평가할 모델 객체.
    dataloader (torch.utils.data.DataLoader): 평가 데이터셋의 DataLoader.
    loss (torch.nn.modules.loss): 손실 함수.
    device (torch.device): 계산에 사용할 장치 (예: 'cuda' 또는 'cpu').

    Returns:
    float: 데이터로더에 있는 모든 데이터에 대한 평균 손실값.
    """
    model.eval() # 모델을 평가 모드로 설정
    total_loss = 0.0

    for input_ids, attention_mask, start_positions, end_positions in dataloader:
        # 데이터를 지정된 장치로 이동
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        start_positions = start_positions.to(device)
        end_positions = end_positions.to(device)

        with torch.no_grad(): # 그라디언트 계산을 비활성화
            # 모델의 예측 결과를 얻음
            start_probs, end_probs = model(input_ids, attention_mask=attention_mask)
            # 시작 위치와 종료 위치에 대한 손실 계산
            loss_start = loss(start_probs, start_positions)
            loss_end = loss(end_probs, end_positions)
            # 손실의 평균을 계산
            batch_loss = (loss_start + loss_end) / 2
            # 총 손실에 배치 손실 추가
            total_loss += batch_loss.item()

    # 평균 손실 계산
    avg_loss = total_loss / len(dataloader)
    return avg_loss

In [ ]:
# BERT 기반의 기계 독해 모델 학습 과정을 실행하는 코드입니다.
for epoch in range(3):  # 총 3 에포크(epoch) 동안 학습을 수행합니다.
    model.train()  # 모델을 학습 모드로 설정합니다.
    total_loss = 0.0  # 에포크별 총 손실을 초기화합니다.

    # tqdm을 사용하여 학습 진행 상황을 시각화하며, train_dataloader의 각 배치(batch)를 순회합니다.
    for input_ids, attention_mask, start_positions, end_positions in tqdm(train_dataloader, total=len(train_dataloader)):
        # 배치 데이터를 현재 설정된 장치(device)로 이동합니다. GPU나 CPU 사용에 따라 달라집니다.
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        start_positions = start_positions.to(device)
        end_positions = end_positions.to(device)

        # 모델에 입력 데이터를 제공하고 시작 및 종료 위치에 대한 예측 확률을 계산합니다.
        start_probs, end_probs = model(input_ids, attention_mask=attention_mask)
        # 손실 함수를 사용하여 예측된 시작 및 종료 위치의 손실을 계산합니다.
        loss_start = loss(start_probs, start_positions)
        loss_end = loss(end_probs, end_positions)

        # 시작 위치와 종료 위치에 대한 손실의 평균을 이 배치의 최종 손실로 계산합니다.
        batch_loss = (loss_start + loss_end) / 2

        optimizer.zero_grad()  # 옵티마이저의 그라디언트를 초기화합니다.
        batch_loss.backward()  # 손실에 대해 역전파를 수행하여 그라디언트를 계산합니다.
        optimizer.step()  # 계산된 그라디언트를 사용하여 모델의 매개변수를 업데이트합니다.

        # 에포크별 총 손실에 이 배치의 손실을 누적합니다.
        total_loss += batch_loss.item()

    # 에포크가 끝날 때마다, 평균 손실을 계산하고 출력합니다.
    avg_loss = total_loss / len(train_dataloader)  # 총 손실을 배치의 수로 나누어 평균 손실을 계산합니다.
    print(f"Epoch {epoch + 1}, Loss: {avg_loss}")  # 현재 에포크의 평균 손실을 출력합니다.

    # 학습된 모델을 평가 모드로 설정하고, 검증 데이터셋에 대한 손실을 계산합니다.
    val_loss = evaluate(model, test_dataloader, loss, device)  # 검증 데이터셋에 대한 평균 손실을 계산하는 함수를 호출합니다.
    print(f"Epoch {epoch + 1}, Val Loss: {val_loss}")  # 검증 데이터셋에 대한 평균 손실을 출력합니다.

100%|██████████| 1092/1092 [08:41<00:00,  2.09it/s]


Epoch 1, Loss: 4.743827617212093
Epoch 1, Val Loss: 4.691907964433942


100%|██████████| 1092/1092 [08:40<00:00,  2.10it/s]


Epoch 2, Loss: 4.687719765600267
Epoch 2, Val Loss: 4.697473326183501


100%|██████████| 1092/1092 [08:39<00:00,  2.10it/s]


Epoch 3, Loss: 4.676241213148767
Epoch 3, Val Loss: 4.692061837514242


In [ ]:
def predict(model, input_ids, attention_mask):
    """
    주어진 입력에 대해 모델이 예측한 시작 인덱스와 종료 인덱스를 반환합니다. 이 함수는 모델을 평가 모드로 설정하고,
    입력 텐서에 대한 예측을 수행한 후, 시작 위치와 종료 위치의 확률 분포에서 가장 높은 확률을 가진 인덱스를 각각 찾아 반환합니다.

    Parameters:
    model (torch.nn.Module): 예측을 수행할 모델 객체.
    input_ids (torch.Tensor): 모델에 입력될 토큰화된 텍스트의 정수 인코딩을 포함하는 텐서.
    attention_mask (torch.Tensor): 입력 텐서에서 실제 텍스트와 패딩을 구분하기 위한 마스크 텐서.

    Returns:
    tuple: (start_index, end_index) 형태로, 모델이 예측한 시작 인덱스와 종료 인덱스를 반환합니다.
    """
    model.eval()  # 모델을 평가 모드로 설정합니다.
    with torch.no_grad():  # 그라디언트 계산을 비활성화합니다.
        # 모델에 입력을 제공하기 전에 입력의 차원을 증가시키고, 계산에 사용할 장치로 이동합니다.
        start_probs, end_probs = model(input_ids.unsqueeze(0).to(device),
                                       attention_mask=attention_mask.unsqueeze(0).to(device))

    # 시작 위치와 종료 위치에 대한 확률 분포에서 가장 높은 확률을 가진 인덱스를 찾습니다.
    start_index = torch.argmax(start_probs).item()  # 시작 인덱스 추출
    end_index = torch.argmax(end_probs).item()  # 종료 인덱스 추출

    return start_index, end_index  # 추출된 시작 및 종료 인덱스를 반환합니다.

In [ ]:
# 테스트 데이터, 출력할 테스트 데이터의 인덱스, 토크나이저를 받아서 해당 인덱스 샘플의 결과를 출력
def display_output(test_data, index, tokenizer):


    # index 번호의 테스트 데이터 샘플을 얻는다.
    input_ids, attention_mask, start_position, end_position = test_data[index]

    #------- 임의의 index로부터 테스트 데이터의 질문(question), 본문(context), 정답(true_answer)을 추출하는 과정 -------#
    # decoded_text는 [CLS] 본문 [SEP] 질문 [SEP]의 형태로 구성된 텍스트.
    decoded_text = tokenizer.decode(input_ids)

    # decoded_text로부터 정답 문자열 추출.
    true_answer = tokenizer.decode(input_ids[start_position:end_position + 1])

    # 본문과 질문 추출하여 각각 context, question에 저장.
    context = decoded_text.split('[SEP]')[0].replace('[CLS]', '').strip()
    question = decoded_text.split('[SEP]')[1].strip()
    #-------------------------------------------------------------------------------------------------------------------#

    # 정수 인코딩 입력과 어텐션 마스크로부터 모델의 예측을 얻는다.
    start_index, end_index = predict(model, input_ids=input_ids, attention_mask=attention_mask)
    predicted_answer = tokenizer.decode(input_ids[start_index:end_index + 1])

    # 본문, 질문, 정답 그리고 모델의 예측을 출력하여 정답과 예측값을 비교한다.
    print(f"본문: {context}")
    print(f"질문: {question}")
    print(f"정답: {true_answer}")
    print(f"예측: {predicted_answer}")

In [ ]:
display_output(test_dataset, 15, tokenizer)

본문: 알렉산더 메이그스 헤이그 2세 ( 영어 : Alexander Meigs Haig, Jr., 1924년 12월 2일 ~ 2010년 2월 20일 ) 는 미국의 국무 장관을 지낸 미국의 군인, 관료 및 정치인이다. 로널드 레이건 대통령 밑에서 국무장관을 지냈으며, 리처드 닉슨과 제럴드 포드 대통령 밑에서 백악관 비서실장을 지냈다. 또한 그는 미국 군대에서 2번째로 높은 직위인 미국 육군 부참모 총장과 나토 및 미국 군대의 유럽연합군 최고사령관이었다. 한국 전쟁 시절 더글러스 맥아더 유엔군 사령관의 참모로 직접 참전하였으며, 로널드 레이건 정부 출범당시 초대 국무장관직을 맡아 1980년대 대한민국과 미국의 관계를 조율해 왔다. 저서로 회고록 《 경고 : 현실주의, 레이건과 외교 정책 》 ( 1984년 발간 ) 이 있다.
질문: 알렉산더 메이그스 헤이그의 생년월일은?
정답: 1924년 12월 2일
예측: 1924년 12월 2일


In [ ]:
index = 5

In [ ]:
display_output(test_dataset, index, tokenizer)

본문: 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받
질문: 임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가?
정답: 여의도 농민 폭력 시위
예측: 여의도 농민 폭력 시위


In [ ]:
for i in range(0, 100):
  display_output(test_dataset, i, tokenizer)
  print('-' * 100)

본문: 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경
질문: 임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?
정답: 1989년 2월 15일
예측: 1989년 2월 15일
----------------------------------------------------------------------------------------------------
본문: 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명